In [1]:
import os
os.environ["NAPARI_ASYNC"] = "1"
#os.environ["NAPARI_PERFMON"] = "/Users/fukai/projects/napari-travali2/examples/prefmon.json"

In [2]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": "/Volumes/Extreme SSD/aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = '/Volumes/Extreme SSD/aligned_image5_image.zarr'
image = ts.open(image_spec).result()[:,0,:,:]
bboxes_df = pd.read_csv('/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv').set_index(["frame","label"])
display(bboxes_df.head())
with open('/Volumes/Extreme SSD/aligned_image5_labels.splits.json') as f:
    splits = json.load(f, object_hook=lambda d: {int(k): [int(_v) for _v in v] for k, v in d.items()})
splits

min_y  min_x  max_y  max_x
frame label                            
0     1         13   1865     57   1909
      2         19   4954     60   4994
      3         29   2832     70   2876
      4         35   1633     67   1669
      5         38   2756     80   2797

{10: [2102, 2103],
 12: [2958, 2959],
 13: [2711, 2710],
 14: [3935, 3936],
 15: [3432, 3431],
 17: [2401, 2400],
 18: [3284, 3283],
 19: [4740, 4743],
 20: [8100, 8099],
 21: [4742, 4744],
 22: [2378, 2379],
 23: [3937, 3938],
 24: [5992, 5993],
 25: [6503, 6504],
 27: [8595, 8596],
 28: [1897, 1896],
 29: [9546, 9547],
 30: [2285, 2286],
 32: [7884, 7885],
 33: [9675, 9674],
 34: [6505, 6506],
 37: [2788, 2789],
 38: [6669, 6668],
 39: [2167, 2166],
 40: [3174, 3173],
 42: [2068, 2067],
 43: [4776, 4777],
 44: [9654, 9655],
 45: [3737, 3736],
 46: [4870, 4869],
 47: [4455, 4456],
 48: [5845, 5844],
 49: [4804, 4805],
 51: [4607, 4606],
 52: [3562, 3561],
 53: [2657, 2658],
 54: [3482, 3481],
 55: [5814, 5815],
 56: [11109, 11110],
 57: [2766, 2767],
 58: [3328, 3329],
 59: [3822, 3823],
 60: [8896, 8897],
 61: [7376, 7375],
 62: [8039, 8040],
 63: [4495, 4494],
 64: [2287, 2290],
 65: [5504, 5503],
 66: [1903, 1900],
 67: [2380, 2381],
 68: [3101, 3100],
 69: [6507, 6508],
 70: [5426

In [3]:
import numpy as np
import tensorstore as ts
import tensorstore_trackarr as tta

print(labels.shape)
ta = tta.TrackArray(labels, splits, {}, bboxes_df)

(1965, 19991, 15437)


In [4]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()

In [ ]:
widget = StateMachineWidget(viewer, ta, image, crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [  253 14602  7174]
INFO:napari_travali2._logging:Region selected: coords [  253 14602  7174]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(13578), np.int64(15626), None), slice(np.int64(6150), np.int64(8198), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(253.0), np.float64(14587.108146997367), np.float64(7173.342475872496))
DEBUG:napari_travali2._logging:data coordinates: [ 253.         1009.108147   1023.34247587]
INFO:napari_travali2._logging:clicked at [ 253 1009 1

INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [   0 8961 5237]
INFO:napari_travali2._logging:Region selected: coords [   0 8961 5237]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(7937), np.int64(9985), None), slice(np.int64(4213), np.int64(6261), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(0.0), np.float64(8944.734322482238), np.float64(5216.96805923945))
DEBUG:napari_travali2._logging:data coordinates: [   0.         1007.73432248 1003.96805924]
INFO:napari_travali2._logging:clicked at [   0 1008 1004] at frame 0 and label value 872
INFO:napari_travali2._logging:Track selected: frame 0 value 872
INFO:napari_travali2._logging:Current daughter candidates count: 0
INFO:napari_travali2._logging:messagebox selected
INFO:napari_travali2._logging:select selected
INFO:napari_travali2._logging:Track